In [1]:
import pandas as pd

# Generation

In [27]:
df = pd.read_csv("intermediate/generation_by_production_type.csv")
df["Grid Node"] = "UK-GB"
df["unit"] = "MWh"
df = df.rename(columns={
    "Biomass (MW)": "BIOMASS",
    "Hydro Pumped Storage (MW)":"HYDRO_PUMPED_STORAGE",
    "Hydro Run-of-river and poundage (MW)": "HYDRO_RUN_OF_RIVER",
    "Fossil Hard coal (MW)": "HARD_COAL",
    "Fossil Gas (MW)": "OTHER_GAS",
    "Fossil Oil (MW)":"OTHER_OIL",
    "Nuclear (MW)":"NUCLEAR",
    "Other (MW)": "OTHER",
    "Wind Offshore (MW)": "WIND_OFF_SHORE",
    "Wind Onshore (MW)":"WIND_ON_SHORE",
    "Solar (MW)":"SOLAR",

})
df.to_csv('generation.csv',index=False)

# Exchanges

In [38]:
name_to_grid_node = {
    "IFA": "FR",
    "IFA2": "FR",
    "Moyle": "UK-NIR",
    "BritNEd": "NL",
    "EastWest": "IE",
    "NEMO": "BE",
    "NorthSeaLink": "NO",
}

In [45]:
df = pd.read_csv('intermediate/interconnector_exchanges.csv')
interconnector_names = list(name_to_grid_node.keys())
df = df[interconnector_names + ['datetime']]
# Pivot to datetime and rows for each interconnector
df = df.melt(id_vars='datetime', var_name='interconnector', value_name='Value')

In [46]:
# Map interconnector names to grid nodes and group by datetime
df.loc[:, "Grid Node"] = df.interconnector.map(name_to_grid_node)
df = df.groupby(['datetime', 'Grid Node']).sum(numeric_only=True).reset_index()
df.loc[:, "unit"] = "MW"

In [47]:
# Exports are when value is < 0
# Imports when it's > 0
df_exports = df[df['Value'] < 0].copy()
df_imports = df[df['Value'] >= 0].copy()

df_exports.loc[:, "Grid Node To"] = "UK-GB"
df_exports = df_exports.rename(columns={"Grid Node": "Grid Node From"})
df_exports.loc[:, "Value"] = df_exports["Value"] * -1
df_exports = df_exports[['datetime', 'Grid Node From', 'Grid Node To', 'Value', 'unit']]

df_imports.loc[:, "Grid Node From"] = "UK-GB"
df_imports = df_imports.rename(columns={"Grid Node": "Grid Node To"})
df_imports = df_imports[['datetime', 'Grid Node From', 'Grid Node To', 'Value', 'unit']]
df = pd.concat([df_exports, df_imports])


In [48]:
# Duplicate all rows and invert the sign of the value
# This is because we want to model the interconnector as a bidirectional flow
df2 = df.copy()
df2.loc[:, "Value"] = df2["Value"] * -1
df2.loc[:, "Grid Node From"] = df["Grid Node To"]
df2.loc[:, "Grid Node To"] = df["Grid Node From"]
df = pd.concat([df, df2])

In [49]:
df.to_csv('imports_exports.csv', index=False)